供应商数据库SPJ中有三张表：

S(SNO, SNAME, STATUS, CITY)

P(PNO, PNAME, COLOR, WEIGHT, CITY)

J(JNO, JNAME,CITY)

SPJ(SNO, PNO, JNO, QTY, PRICE)

S表示供应商，各属性依次为供应商号，供应商名，供应商状态值，供应商所在城市；

P表示零件，各属性依次为零件号，零件名，零件颜色，零件重量，零件存放的城市；

J表示工程，各属性依次为工程号，工程名，工程所在城市；

SPJ表示供货关系，各属性依次为供应商号，零件号，工程号，供货数量，单价。

1. 求没有供应零件号为P1和P2两种零件的供应商姓名。

In [ ]:
SELECT S.SNAME
FROM S
WHERE S.SNO NOT IN (
    SELECT SPJ.SNO
    FROM SPJ
    WHERE SPJ.PNO IN ('P1', 'P2')
);

2. 列出所有供应商的信息，包括供应商姓名、所供应的零件名（没有供应零件的供应商也要列出，最后结果中不要出现重复元组）

In [ ]:
SELECT DISTINCT S.SNAME, P.PNAME
FROM S
LEFT JOIN SPJ ON S.SNO = SPJ.SNO
LEFT JOIN P ON SPJ.PNO = P.PNO;

3. 求只向与自己位于不同城市的工程供应零件的供应商姓名。

In [ ]:
SELECT DISTINCT S.SNAME
FROM S
JOIN SPJ ON S.SNO = SPJ.SNO
JOIN J ON SPJ.JNO = J.JNO
WHERE S.CITY <> J.CITY
AND S.SNO NOT IN (
    SELECT S2.SNO
    FROM S AS S2
    JOIN SPJ AS SPJ2 ON S2.SNO = SPJ2.SNO
    JOIN J AS J2 ON SPJ2.JNO = J2.JNO
    WHERE S2.CITY = J2.CITY
);

4. 求只向与自己位于相同城市的工程供应零件的供应商姓名。

In [ ]:
SELECT DISTINCT S.SNAME
FROM S
JOIN SPJ ON S.SNO = SPJ.SNO
JOIN J ON SPJ.JNO = J.JNO
WHERE S.CITY = J.CITY
AND S.SNO NOT IN (
    SELECT S2.SNO
    FROM S AS S2
    JOIN SPJ AS SPJ2 ON S2.SNO = SPJ2.SNO
    JOIN J AS J2 ON SPJ2.JNO = J2.JNO
    WHERE S2.CITY <> J2.CITY
);

5. 求供应了所有零件的供应商姓名。
   1. 在子查询中，我们找出在供货关系表SPJ中每个供应商（SPJ.SNO）所供应的不同零件数目。使用GROUP BY和COUNT(DISTINCT SPJ.PNO)来计算每个供应商供应的不同零件的数量。
   2. 通过使用HAVING子句，我们筛选出那些供应了与所有零件数目相同的供应商。这里的子查询SELECT COUNT(*) FROM P用于获取所有零件的总数。
   3. 最后，我们将筛选出的供应商编号（SPJ.SNO）与供应商表S中的供应商编号进行匹配，以获取供应商的姓名（S.SNAME）。

In [ ]:
SELECT S.SNAME
FROM S
WHERE S.SNO IN (
    SELECT SPJ.SNO
    FROM SPJ
    GROUP BY SPJ.SNO
    HAVING COUNT(DISTINCT SPJ.PNO) = (
        SELECT COUNT(*) FROM P
    )
);

6. 求供应了所有红色零件的供应商姓名。

In [ ]:
SELECT S.SNAME
FROM S
JOIN SPJ ON S.SNO = SPJ.SNO
WHERE SPJ.PNO IN (SELECT PNO FROM P WHERE COLOR = '红色')
GROUP BY S.SNAME
HAVING COUNT(DISTINCT SPJ.PNO) = (SELECT COUNT(*) FROM P WHERE COLOR = '红色');

7. 列出每个城市的工程所使用的零件总的数量。

In [ ]:
SELECT J.CITY, SUM(SPJ.QTY)
FROM J
JOIN SPJ ON J.JNO = SPJ.JNO
GROUP BY J.CITY;

8. 按零件数量总和的降序列出每项工程所使用的每种红色零件的总的金额（工程可以向不同的供应商购买同一零件，总金额＝单价*供货数量）。输出工程号、零件号、总金额。

In [ ]:
SELECT SPJ.JNO, SPJ.PNO, SUM(SPJ.QTY * SPJ.PRICE) AS TOTAL_AMOUNT
FROM SPJ
JOIN P ON SPJ.PNO = P.PNO
WHERE P.COLOR = '红色'
GROUP BY SPJ.JNO, SPJ.PNO
ORDER BY SUM(SPJ.QTY) DESC;

9. 求供应零件数量最多的供应商姓名。

In [ ]:
SELECT S.SNAME
FROM S
JOIN SPJ ON S.SNO = SPJ.SNO
GROUP BY S.SNAME
ORDER BY SUM(SPJ.QTY) DESC
LIMIT 1;

10. 求每个城市中供应零件数量最多的供应商姓名。

In [ ]:
WITH CitySupplies AS (
    SELECT 
        S.CITY, 
        S.SNAME, 
        SUM(SPJ.QTY) AS TotalQuantity
    FROM 
        S JOIN SPJ ON S.SNO = SPJ.SNO
    GROUP BY 
        S.CITY, 
        S.SNAME
),
RankedSuppliers AS (
    SELECT 
        CITY, 
        SNAME, 
        TotalQuantity,
        RANK() OVER (PARTITION BY CITY ORDER BY TotalQuantity DESC) AS Ranking
    FROM 
        CitySupplies
)
SELECT 
    CITY, 
    SNAME
FROM 
    RankedSuppliers
WHERE 
    Ranking = 1;

11. 列出恰好供应了相同零件的供应商对。

注意：输出的时候，供应商对小号在前，相等不输出，要去重，都没有供应零件也算供应了相同零件。输出形式：(SNO,SNO)

In [ ]:
SELECT DISTINCT
    A.SNO AS SNO1,
    B.SNO AS SNO2
FROM
    (SELECT S.SNO, GROUP_CONCAT(DISTINCT IFNULL(SPJ.PNO, 'NONE') ORDER BY SPJ.PNO) AS PNO_LIST
     FROM S
     LEFT JOIN SPJ ON S.SNO = SPJ.SNO
     GROUP BY S.SNO) A
JOIN
    (SELECT S.SNO, GROUP_CONCAT(DISTINCT IFNULL(SPJ.PNO, 'NONE') ORDER BY SPJ.PNO) AS PNO_LIST
     FROM S
     LEFT JOIN SPJ ON S.SNO = SPJ.SNO
     GROUP BY S.SNO) B
ON
    A.PNO_LIST = B.PNO_LIST AND A.SNO < B.SNO
ORDER BY
    SNO1, SNO2;